In [9]:
from dataclasses import dataclass
from statistics import mean, median, stdev
from numpy import sin, pi, exp
from tqdm import tqdm
import requests
import pandas as pd
import itertools
import collections
import random

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [10]:
bootstrap_static = requests.get('https://fantasy.premierleague.com/api/bootstrap-static/').json()

In [11]:
def team_name(team_code):
    for team in bootstrap_static['teams']:
        if team['code'] == team_code:
            return team['short_name']

def position(element_type_id):
    for element_type in bootstrap_static['element_types']:
        if element_type['id'] == element_type_id:
            return element_type['singular_name_short']

def squad_select(position):
    for element_type in bootstrap_static['element_types']:
        if element_type['singular_name_short'] == position:
            return element_type['squad_select']


In [12]:
pool_pd = pd.DataFrame.from_dict(bootstrap_static['elements'])

pool_pd['position'] = pool_pd['element_type'].apply(position)
pool_pd['team'] = pool_pd['team_code'].apply(team_name)
pool_pd['selected_by_percent'] = pool_pd['selected_by_percent'].apply(float)

# Has played 25%
#pool_pd = pool_pd.loc[pool_pd['minutes'] > 0]
#pool_pd = pool_pd.loc[pool_pd['selected_by_percent'] > 0]

# News is only a bad sign, like transfers or injuries.
pool_pd = pool_pd.loc[pool_pd['news'].apply(len) == 0]

def norm(rows):
    _min = rows.min()
    _max = rows.max()
    return (rows - _min) / (_max - _min)


def sigmoid(x):
    return 1 / (1 + exp(-x))


pool_pd['score'] = (
    sin(norm(pool_pd.total_points) * pi / 2) *
    sin(norm(pool_pd.minutes) * pi / 2) *
    sin(norm(pool_pd.selected_by_percent) * pi / 2))

pool_pd['score'] = (
    sigmoid(norm(pool_pd.total_points)) *
    sigmoid(norm(pool_pd.minutes)) *
    sigmoid(norm(pool_pd.selected_by_percent)))
#pool_pd['score'] = pool_pd.total_points
#acc = lambda x: (min(x) + 100) 
#acc = lambda x: min(x) + 0.05
acc = mean

# Only pick candidates that are above averge in their position.
pool_gkp = pool_pd[pool_pd['position'] == 'GKP']
pool_gkp = pool_gkp.loc[pool_gkp['score'] > acc(pool_gkp['score'])]

pool_def = pool_pd[pool_pd['position'] == 'DEF']
pool_def = pool_def.loc[pool_def['score'] > acc(pool_def['score'])]

pool_mid = pool_pd[pool_pd['position'] == 'MID']
pool_mid = pool_mid.loc[pool_mid['score'] > acc(pool_mid['score'])]

pool_fwd = pool_pd[pool_pd['position'] == 'FWD']
pool_fwd = pool_fwd.loc[pool_fwd['score'] > acc(pool_fwd['score'])]

pool_pd = pd.concat([pool_gkp, pool_def, pool_mid, pool_fwd])
#pool_pd = pool_pd.loc[pool_pd['score'] > 0.005]


pool_pd['score'] = pool_pd.score.apply(lambda x: round(x, 5))
pool_pd = pool_pd.sort_values('score', ascending=False)

pool_pd = pool_pd[['now_cost', 'total_points', 'web_name', 'minutes', 'team', 'score', 'position']]

print(f'Players in pool: {len(pool_pd)}')

pool_pd.head(10)

Players in pool: 186


,now_cost,total_points,web_name,minutes,team,score,position
319,120,244,Fernandes,3101,MUN,0.37023,MID
271,125,231,Salah,3077,LIV,0.36870,MID
33,55,186,Martínez,3420,AVL,0.32661,GKP
317,55,124,Shaw,2653,MUN,0.31257,DEF
423,100,228,Son,3119,TOT,0.30384,MID
275,75,160,Alexander-Arnold,3029,LIV,0.30055,DEF
245,80,194,Bamford,3052,LEE,0.28492,FWD
244,55,171,Dallas,3410,LEE,0.27857,MID
302,60,142,Dias,2843,MCI,0.27373,DEF
56,80,131,Ings,2172,AVL,0.27321,FWD


In [13]:
@dataclass(frozen=True, eq=True)
class Player:
    name: str
    team: str
    position: str
    cost: int
    score: float
    points: int
        
def flatten(t):
    return [item for sublist in t for item in sublist]
        
def cost(lineup):
    return sum(player.cost for player in lineup)

def score(lineup):
    return sum(player.score for player in lineup)

def positions():
    yield from ('GKP', 'DEF', 'MID', 'FWD')
    
def xPtt(lineup):
    return sum(p.points for p in lineup)

players = [
    Player(row.web_name,
           row.team,
           row.position,
           row.now_cost,
           row.score,
           row.total_points,
          )
    for _, row in pool_pd.iterrows()
]

def keep_top_n_per_position(lineup, p_cutoff):
    new = []
    # Keep the top n for every position
    for position, cutoff in p_cutoff:
        candidates = (p for p in players if p.position == position)
        new.extend(sorted(candidates, key=lambda p: p.score, reverse=True)[:cutoff])
    return new

def grp_by_cost(lineup):
    grp = collections.defaultdict(list)
    for p in lineup:
        grp[p.cost].append(p)
    return grp

def top_n_score_by_cost(lineup, n=3):        
    return [sorted(v, key=lambda v: v.score, reverse=True)[:n] for v in grp_by_cost(lineup).values()]

def top_n_score_by_cost_by_positions(lineup, cutoff=(2, 3, 3, 2)):
    new = []
    for n, pos in zip(cutoff, positions()):
        d = top_n_score_by_cost((p for p in players if p.position == pos), n=n)
        new.extend(flatten(d))
    return new

def pprint(pool):
    for pos in positions():
        pos_players = sorted((p for p in pool if p.position == pos), key=lambda p: p.score, reverse=True)
        print(f'{pos}(n={len(pos_players)}, score={score(pos_players)}, cost={cost(pos_players)})')
        for player in pos_players:
            print(f' {player}')

def sprint(pool):
    print(f'score: {score(pool)}, cost: {cost(pool)}, xP(TP): {xPtt(pool)}')
    pprint(pool)
    
    
def drop_names(pool, remove):
    return [p for p in pool if p.name not in remove]

#players = drop_names(players, ('Lingard',))
players = top_n_score_by_cost_by_positions(players)

print(f'Players in pool: {len(players)}')
pprint(players)
    

Players in pool: 67
GKP(n=8, score=2.05868, cost=420)
 Player(name='Martínez', team='AVL', position='GKP', cost=55, score=0.32661, points=186)
 Player(name='Schmeichel', team='LEI', position='GKP', cost=50, score=0.26343, points=128)
 Player(name='Ederson', team='MCI', position='GKP', cost=60, score=0.25876, points=160)
 Player(name='Sánchez', team='BHA', position='GKP', cost=45, score=0.24692, points=101)
 Player(name='Lloris', team='TOT', position='GKP', cost=55, score=0.24417, points=149)
 Player(name='Meslier', team='LEE', position='GKP', cost=50, score=0.24224, points=154)
 Player(name='Mendy', team='CHE', position='GKP', cost=60, score=0.23882, points=140)
 Player(name='Guaita', team='CRY', position='GKP', cost=45, score=0.23773, points=124)
DEF(n=13, score=3.35924, cost=705)
 Player(name='Shaw', team='MUN', position='DEF', cost=55, score=0.31257, points=124)
 Player(name='Alexander-Arnold', team='LIV', position='DEF', cost=75, score=0.30055, points=160)
 Player(name='Dias', team

In [14]:
def team_constraint(lineup):
    count = collections.Counter(p.team for p in lineup)
    return max(count.values()) <= 2

def position_constraint(lineup, n, position):
    count = collections.Counter(p.team for p in lineup if p.position == position)
    return max(count.values()) <= n

def gkp_def_not_same_team(lineup):
    count = collections.Counter(p.team for p in lineup if p.position == 'GKP' or p.position == 'DEF')
    return max(count.values()) <= 1

def must_contain(lineup, must):
    if must and lineup:
        return must.issubset(set(p.name for p in lineup))
    return True

In [19]:
def lineup(buget=1_000, base=((), (), (), (),)):

    _gkp = sorted((p for p in players if p.position == 'GKP'), key=lambda p: p.score, reverse=True)
    _def = sorted((p for p in players if p.position == 'DEF'), key=lambda p: p.score, reverse=True)
    _mid = sorted((p for p in players if p.position == 'MID'), key=lambda p: p.score, reverse=True)
    _fwd = sorted((p for p in players if p.position == 'FWD'), key=lambda p: p.score, reverse=True)
    
    m_gkps, m_defs, m_mids, m_fwds = [set(m) for m in base]
    
    def _gkp_combinations():
         yield from filter(
            lambda g: position_constraint(g, 1, 'GKP') and must_contain(g, m_gkps),
            itertools.combinations(_gkp, 2)
        )

    def _def_combinations():
         yield from filter(
            lambda d: position_constraint(d, 1, 'DEF') and must_contain(d, m_defs),
            itertools.combinations(_def, 5)
        )
    
    def _mid_combinations():
         yield from filter(
            lambda m: position_constraint(m, 2, 'MID') and must_contain(m, m_mids),
            itertools.combinations(_mid, 5)
        )

    def _fwd_combinations():
        yield from filter(
            lambda f: position_constraint(f, 1, 'FWD') and must_contain(f, m_fwds),
            itertools.combinations(_fwd, 3)
        )
        
    gkp_combinations = tuple(sorted(_gkp_combinations(), key=score, reverse=False))
    def_combinations = tuple(sorted(_def_combinations(), key=score, reverse=False))
    mid_combinations = tuple(sorted(_mid_combinations(), key=score, reverse=False))
    fwd_combinations = tuple(sorted(_fwd_combinations(), key=score, reverse=False))
    
    min_cost_def = cost(min(gkp_combinations, key=cost))
    min_cost_mid = cost(min(mid_combinations, key=cost))
    min_cost_fwd = cost(min(fwd_combinations, key=cost))

    max_cost_def = cost(max(gkp_combinations, key=cost))
    max_cost_mid = cost(max(mid_combinations, key=cost))
    max_cost_fwd = cost(max(fwd_combinations, key=cost))

    max_score_gkp = score(max(gkp_combinations, key=score))
    max_score_def = score(max(def_combinations, key=score))
    max_score_mid = score(max(mid_combinations, key=score))
    max_score_fwd = score(max(fwd_combinations, key=score))
    
    min_score_def = score(min(def_combinations, key=score))
    min_score_mid = score(min(mid_combinations, key=score))
    min_score_fwd = score(min(fwd_combinations, key=score))

    min_cost_mid_fwd = min_cost_mid + min_cost_fwd
    min_cost_def_mid_fwd = min_cost_def + min_cost_mid + min_cost_fwd
    
    max_cost_mid_fwd = max_cost_mid + max_cost_fwd
    max_cost_def_mid_fwd = max_cost_def + max_cost_mid + max_cost_fwd

    max_score_mid_fwd = max_score_mid + max_score_fwd
    max_score_def_mid_fwd = max_score_def + max_score_mid + max_score_fwd
    
    min_score_mid_fwd = min_score_mid + min_score_fwd
    min_score_def_mid_fwd = min_score_def + min_score_mid + min_score_fwd

    best_score = 0
    best_lineup = tuple()
    buget_lower = buget * .90
    score_lower = (max_score_gkp + max_score_def + max_score_mid + max_score_fwd) * .90

    print(f'{min_cost_mid=}, {min_cost_fwd=}, {min_cost_mid_fwd=}')
    print(f'{max_score_mid=}, {max_score_fwd=}, {max_score_mid_fwd=}')
    print(f'{m_gkps=}, {m_defs=}, {m_mids=}, {m_fwds=}')
    print(f'{score_lower=}')
    
    def lvl0(c):
        return (buget_lower - max_cost_def_mid_fwd <= cost(c) <= buget - min_cost_def_mid_fwd
            and score(c) + max_score_def_mid_fwd > best_score
            and score(c) + max_score_def_mid_fwd > score_lower
            and team_constraint(c))

    def lvl1(c):
        return (buget_lower - max_cost_mid_fwd  <= cost(c) <= buget - min_cost_mid_fwd
            and score(c) + max_score_mid_fwd > best_score
            and score(c) + max_score_mid_fwd > score_lower
            and team_constraint(c)
            and gkp_def_not_same_team(c))

    def lvl2(c):
        return (buget_lower - max_cost_fwd <= cost(c) <= buget - min_cost_fwd
            and score(c) + max_score_fwd > best_score
            and score(c) + max_score_fwd > score_lower
            and team_constraint(c))

    def lvl3(c):
        return (cost(c) <= buget
            and score(c) > best_score
            and score(c) > score_lower
            and team_constraint(c))

    total = len(gkp_combinations) * len(def_combinations) * len(mid_combinations) * len(fwd_combinations)
    step = len(mid_combinations) * len(fwd_combinations)
    
    with tqdm(
        total=total,
        bar_format='{desc:<5.5}{percentage:3.0f}%|{bar:20}{r_bar}',
        unit_scale=True,
        unit_divisor=2**10,
    ) as bar:
         for g in gkp_combinations:
            if lvl0(g):
                for d in def_combinations:
                    bar.update(step)
                    g1 = g + d
                    if lvl1(g1):
                        for m in mid_combinations:
                            g2 = g1 + m
                            if lvl2(g2):
                                for f in fwd_combinations:
                                    g3 = g2 + f
                                    if lvl3(g3):
                                        best_score = score(g3)
                                        best_lineup = g3
                                        print('-' * 100)
                                        sprint(g3)
    
    return best_lineup

base = (
    (),
    ('Alexander-Arnold', 'Shaw', 'Dias'),
    ('Salah',),
    (),
)

l = lineup(base=base)
sprint(l)

min_cost_mid=310, min_cost_fwd=175, min_cost_mid_fwd=485
max_score_mid=1.58923, max_score_fwd=0.82672, max_score_mid_fwd=2.41595
m_gkps=set(), m_defs={'Dias', 'Shaw', 'Alexander-Arnold'}, m_mids={'Salah'}, m_fwds=set()
score_lower=3.987225


       0%|                    | 0.00/8.09G [00:00<?, ?it/s]

----------------------------------------------------------------------------------------------------
score: 3.9873500000000006, cost: 1000, xP(TP): 2082
GKP(n=2, score=0.47655000000000003, cost=105)
 Player(name='Mendy', team='CHE', position='GKP', cost=60, score=0.23882, points=140)
 Player(name='Guaita', team='CRY', position='GKP', cost=45, score=0.23773, points=124)
DEF(n=5, score=1.36361, cost=280)
 Player(name='Shaw', team='MUN', position='DEF', cost=55, score=0.31257, points=124)
 Player(name='Alexander-Arnold', team='LIV', position='DEF', cost=75, score=0.30055, points=160)
 Player(name='Dias', team='MCI', position='DEF', cost=60, score=0.27373, points=142)
 Player(name='Ayling', team='LEE', position='DEF', cost=45, score=0.24234, points=100)
 Player(name='Coady', team='WOL', position='DEF', cost=45, score=0.23442, points=106)
MID(n=5, score=1.32047, cost=350)
 Player(name='Salah', team='LIV', position='MID', cost=125, score=0.3687, points=231)
 Player(name='Mount', team='CHE', 

----------------------------------------------------------------------------------------------------
score: 4.00443, cost: 995, xP(TP): 2104
GKP(n=2, score=0.47655000000000003, cost=105)
 Player(name='Mendy', team='CHE', position='GKP', cost=60, score=0.23882, points=140)
 Player(name='Guaita', team='CRY', position='GKP', cost=45, score=0.23773, points=124)
DEF(n=5, score=1.36361, cost=280)
 Player(name='Shaw', team='MUN', position='DEF', cost=55, score=0.31257, points=124)
 Player(name='Alexander-Arnold', team='LIV', position='DEF', cost=75, score=0.30055, points=160)
 Player(name='Dias', team='MCI', position='DEF', cost=60, score=0.27373, points=142)
 Player(name='Ayling', team='LEE', position='DEF', cost=45, score=0.24234, points=100)
 Player(name='Coady', team='WOL', position='DEF', cost=45, score=0.23442, points=106)
MID(n=5, score=1.36651, cost=375)
 Player(name='Salah', team='LIV', position='MID', cost=125, score=0.3687, points=231)
 Player(name='Mount', team='CHE', position='MI

----------------------------------------------------------------------------------------------------
score: 4.01534, cost: 1000, xP(TP): 2071
GKP(n=2, score=0.47655000000000003, cost=105)
 Player(name='Mendy', team='CHE', position='GKP', cost=60, score=0.23882, points=140)
 Player(name='Guaita', team='CRY', position='GKP', cost=45, score=0.23773, points=124)
DEF(n=5, score=1.36361, cost=280)
 Player(name='Shaw', team='MUN', position='DEF', cost=55, score=0.31257, points=124)
 Player(name='Alexander-Arnold', team='LIV', position='DEF', cost=75, score=0.30055, points=160)
 Player(name='Dias', team='MCI', position='DEF', cost=60, score=0.27373, points=142)
 Player(name='Ayling', team='LEE', position='DEF', cost=45, score=0.24234, points=100)
 Player(name='Coady', team='WOL', position='DEF', cost=45, score=0.23442, points=106)
MID(n=5, score=1.39037, cost=385)
 Player(name='Fernandes', team='MUN', position='MID', cost=120, score=0.37023, points=244)
 Player(name='Salah', team='LIV', positi

...

----------------------------------------------------------------------------------------------------
score: 4.203720000000001, cost: 1000, xP(TP): 2262
GKP(n=2, score=0.57353, cost=100)
 Player(name='Martínez', team='AVL', position='GKP', cost=55, score=0.32661, points=186)
 Player(name='Sánchez', team='BHA', position='GKP', cost=45, score=0.24692, points=101)
DEF(n=5, score=1.40367, cost=285)
 Player(name='Shaw', team='MUN', position='DEF', cost=55, score=0.31257, points=124)
 Player(name='Alexander-Arnold', team='LIV', position='DEF', cost=75, score=0.30055, points=160)
 Player(name='Dias', team='MCI', position='DEF', cost=60, score=0.27373, points=142)
 Player(name='White', team='ARS', position='DEF', cost=45, score=0.26615, points=104)
 Player(name='Coufal', team='WHU', position='DEF', cost=50, score=0.25067, points=128)
MID(n=5, score=1.44171, cost=385)
 Player(name='Salah', team='LIV', position='MID', cost=125, score=0.3687, points=231)
 Player(name='Son', team='TOT', position='M

     100%|███████████████████▉| 8.06G/8.09G [01:45<00:00, 67.3Mit/s]

----------------------------------------------------------------------------------------------------
score: 4.207140000000001, cost: 995, xP(TP): 2242
GKP(n=2, score=0.59004, cost=105)
 Player(name='Martínez', team='AVL', position='GKP', cost=55, score=0.32661, points=186)
 Player(name='Schmeichel', team='LEI', position='GKP', cost=50, score=0.26343, points=128)
DEF(n=5, score=1.4242599999999999, cost=290)
 Player(name='Shaw', team='MUN', position='DEF', cost=55, score=0.31257, points=124)
 Player(name='Alexander-Arnold', team='LIV', position='DEF', cost=75, score=0.30055, points=160)
 Player(name='Dias', team='MCI', position='DEF', cost=60, score=0.27373, points=142)
 Player(name='Cresswell', team='WHU', position='DEF', cost=55, score=0.27126, points=153)
 Player(name='White', team='ARS', position='DEF', cost=45, score=0.26615, points=104)
MID(n=5, score=1.36612, cost=335)
 Player(name='Salah', team='LIV', position='MID', cost=125, score=0.3687, points=231)
 Player(name='Dallas', team

     100%|████████████████████| 8.09G/8.09G [01:46<00:00, 81.8Mit/s]

score: 4.214160000000001, cost: 990, xP(TP): 2218
GKP(n=2, score=0.59004, cost=105)
 Player(name='Martínez', team='AVL', position='GKP', cost=55, score=0.32661, points=186)
 Player(name='Schmeichel', team='LEI', position='GKP', cost=50, score=0.26343, points=128)
DEF(n=5, score=1.4242599999999999, cost=290)
 Player(name='Shaw', team='MUN', position='DEF', cost=55, score=0.31257, points=124)
 Player(name='Alexander-Arnold', team='LIV', position='DEF', cost=75, score=0.30055, points=160)
 Player(name='Dias', team='MCI', position='DEF', cost=60, score=0.27373, points=142)
 Player(name='Cresswell', team='WHU', position='DEF', cost=55, score=0.27126, points=153)
 Player(name='White', team='ARS', position='DEF', cost=45, score=0.26615, points=104)
MID(n=5, score=1.40576, cost=360)
 Player(name='Salah', team='LIV', position='MID', cost=125, score=0.3687, points=231)
 Player(name='Dallas', team='LEE', position='MID', cost=55, score=0.27857, points=171)
 Player(name='Mount', team='CHE', positio

## Best sin(x)

```
score: 3.9234999999999998, cost: 1000, xP(TP): 2182
GKP(n=2, score=0.25573, cost=100)
 Player(name='Martínez', team='AVL', position='GKP', cost=55, score=0.22351, points=186)
 Player(name='Sánchez', team='BHA', position='GKP', cost=45, score=0.03222, points=101)
DEF(n=5, score=0.69791, cost=270)
 Player(name='Alexander-Arnold', team='LIV', position='DEF', cost=75, score=0.40852, points=160)
 Player(name='Shaw', team='MUN', position='DEF', cost=55, score=0.18395, points=124)
 Player(name='Coufal', team='WHU', position='DEF', cost=50, score=0.0547, points=128)
 Player(name='White', team='ARS', position='DEF', cost=45, score=0.03779, points=104)
 Player(name='Coady', team='WOL', position='DEF', cost=45, score=0.01295, points=106)
MID(n=5, score=2.08969, cost=395)
 Player(name='Salah', team='LIV', position='MID', cost=125, score=0.97841, points=231)
 Player(name='Fernandes', team='MUN', position='MID', cost=120, score=0.97471, points=244)
 Player(name='Dallas', team='LEE', position='MID', cost=55, score=0.11102, points=171)
 Player(name='Bissouma', team='BHA', position='MID', cost=45, score=0.01559, points=77)
 Player(name='Højbjerg', team='TOT', position='MID', cost=50, score=0.00996, points=107)
FWD(n=3, score=0.8801699999999999, cost=235)
 Player(name='Ings', team='AVL', position='FWD', cost=80, score=0.34257, points=131)
 Player(name='Bamford', team='LEE', position='FWD', cost=80, score=0.34251, points=194)
 Player(name='Antonio', team='WHU', position='FWD', cost=75, score=0.19509, points=118)
```

### Total best
```
score: 2474, cost: 1000, xP(TP): 8461080
GKP(n=2, score=346, cost=115)
 Player(name='Martínez', team='AVL', position='GKP', cost=55, score=186)
 Player(name='Ederson', team='MCI', position='GKP', cost=60, score=160)
DEF(n=5, score=707, cost=270)
 Player(name='Cresswell', team='WHU', position='DEF', cost=55, score=153)
 Player(name='Wan-Bissaka', team='MUN', position='DEF', cost=55, score=144)
 Player(name='Dias', team='MCI', position='DEF', cost=60, score=142)
 Player(name='Targett', team='AVL', position='DEF', cost=50, score=138)
 Player(name='Dunk', team='BHA', position='DEF', cost=50, score=130)
MID(n=5, score=847, cost=340)
 Player(name='Son', team='TOT', position='MID', cost=100, score=228)
 Player(name='Dallas', team='LEE', position='MID', cost=55, score=171)
 Player(name='Harrison', team='LEE', position='MID', cost=60, score=160)
 Player(name='Soucek', team='WHU', position='MID', cost=60, score=147)
 Player(name='Bowen', team='WHU', position='MID', cost=65, score=141)
FWD(n=3, score=574, cost=275)
 Player(name='Kane', team='TOT', position='FWD', cost=125, score=242)
 Player(name='Bamford', team='LEE', position='FWD', cost=80, score=194)
 Player(name='Wood', team='BUR', position='FWD', cost=70, score=138)
 ```
 

## Best so far
```
score: 1.000252, cost: 1000, xP(TP): 3420.8618399999996
GKP(n=2, score=0.103275, cost=105)
 Player(name='Martínez', team='AVL', position='GKP', cost=55, score=0.054386)
 Player(name='Meslier', team='LEE', position='GKP', cost=50, score=0.048889)
DEF(n=5, score=0.303172, cost=275)
 Player(name='Mendy', team='MCI', position='DEF', cost=55, score=0.066246)
 Player(name='Stones', team='MCI', position='DEF', cost=55, score=0.066218)
 Player(name='Chilwell', team='CHE', position='DEF', cost=60, score=0.060805)
 Player(name='Alonso', team='CHE', position='DEF', cost=55, score=0.059499)
 Player(name='Coleman', team='EVE', position='DEF', cost=50, score=0.050404)
MID(n=5, score=0.369966, cost=355)
 Player(name='Jota', team='LIV', position='MID', cost=75, score=0.077758)
 Player(name='Gündogan', team='MCI', position='MID', cost=75, score=0.077531)
 Player(name='Lingard', team='MUN', position='MID', cost=70, score=0.074491)
 Player(name='Pépé', team='ARS', position='MID', cost=75, score=0.070984)
 Player(name='El Ghazi', team='AVL', position='MID', cost=60, score=0.069202)
FWD(n=3, score=0.223839, cost=265)
 Player(name='Kane', team='TOT', position='FWD', cost=125, score=0.078495)
 Player(name='Iheanacho', team='LEI', position='FWD', cost=75, score=0.075758)
 Player(name='Rodrigo', team='LEE', position='FWD', cost=65, score=0.069586)
```

## HOF
```
score: 0.973768, cost: 1000, xP(TP): 3330.2865599999996
GKP(n=2, score=0.105388, cost=115)
 Player(name='Martínez', team='AVL', position='GKP', cost=55, score=0.054386)
 Player(name='Mendy', team='CHE', position='GKP', cost=60, score=0.051002)
DEF(n=5, score=0.274034, cost=260)
 Player(name='Mendy', team='MCI', position='DEF', cost=55, score=0.066246)
 Player(name='Chilwell', team='CHE', position='DEF', cost=60, score=0.060805)
 Player(name='Coleman', team='EVE', position='DEF', cost=50, score=0.050404)
 Player(name='Aurier', team='TOT', position='DEF', cost=50, score=0.049844)
 Player(name='Phillips', team='LIV', position='DEF', cost=45, score=0.046735)
MID(n=5, score=0.376421, cost=400)
 Player(name='Fernandes', team='MUN', position='MID', cost=120, score=0.078684)
 Player(name='Jota', team='LIV', position='MID', cost=75, score=0.077758)
 Player(name='Gündogan', team='MCI', position='MID', cost=75, score=0.077531)
 Player(name='Torres', team='MCI', position='MID', cost=70, score=0.073246)
 Player(name='El Ghazi', team='AVL', position='MID', cost=60, score=0.069202)
FWD(n=3, score=0.217925, cost=225)
 Player(name='Iheanacho', team='LEI', position='FWD', cost=75, score=0.075758)
 Player(name='Cavani', team='MUN', position='FWD', cost=85, score=0.072581)
 Player(name='Rodrigo', team='LEE', position='FWD', cost=65, score=0.069586)
```


### Salah + Arnold
```
score: 0.96717, cost: 1000, xP(TP): 3307.7214
GKP(n=2, score=0.103275, cost=105)
 Player(name='Martínez', team='AVL', position='GKP', cost=55, score=0.054386)
 Player(name='Meslier', team='LEE', position='GKP', cost=50, score=0.048889)
DEF(n=5, score=0.278816, cost=285)
 Player(name='Mendy', team='MCI', position='DEF', cost=55, score=0.066246)
 Player(name='Alonso', team='CHE', position='DEF', cost=55, score=0.059499)
 Player(name='Alexander-Arnold', team='LIV', position='DEF', cost=75, score=0.052823)
 Player(name='Coleman', team='EVE', position='DEF', cost=50, score=0.050404)
 Player(name='Aurier', team='TOT', position='DEF', cost=50, score=0.049844)
MID(n=5, score=0.37281, cost=405)
 Player(name='Jota', team='LIV', position='MID', cost=75, score=0.077758)
 Player(name='Gündogan', team='MCI', position='MID', cost=75, score=0.077531)
 Player(name='Salah', team='LIV', position='MID', cost=125, score=0.075073)
 Player(name='Torres', team='MCI', position='MID', cost=70, score=0.073246)
 Player(name='El Ghazi', team='AVL', position='MID', cost=60, score=0.069202)
FWD(n=3, score=0.212269, cost=205)
 Player(name='Iheanacho', team='LEI', position='FWD', cost=75, score=0.075758)
 Player(name='Rodrigo', team='LEE', position='FWD', cost=65, score=0.069586)
 Player(name='Abraham', team='CHE', position='FWD', cost=65, score=0.066925)
 ```
 
 ### Salah + Arond + max 2 from a team
 ```
 score: 0.954856, cost: 1000, xP(TP): 3265.6075200000005
GKP(n=2, score=0.104386, cost=110)
 Player(name='Martínez', team='AVL', position='GKP', cost=55, score=0.054386)
 Player(name='Pope', team='BUR', position='GKP', cost=55, score=0.05)
DEF(n=5, score=0.280122, cost=290)
 Player(name='Mendy', team='MCI', position='DEF', cost=55, score=0.066246)
 Player(name='Chilwell', team='CHE', position='DEF', cost=60, score=0.060805)
 Player(name='Alexander-Arnold', team='LIV', position='DEF', cost=75, score=0.052823)
 Player(name='Coleman', team='EVE', position='DEF', cost=50, score=0.050404)
 Player(name='Aurier', team='TOT', position='DEF', cost=50, score=0.049844)
MID(n=5, score=0.358079, cost=395)
 Player(name='Gündogan', team='MCI', position='MID', cost=75, score=0.077531)
 Player(name='Salah', team='LIV', position='MID', cost=125, score=0.075073)
 Player(name='Pépé', team='ARS', position='MID', cost=75, score=0.070984)
 Player(name='El Ghazi', team='AVL', position='MID', cost=60, score=0.069202)
 Player(name='Willock', team='ARS', position='MID', cost=60, score=0.065289)
FWD(n=3, score=0.212269, cost=205)
 Player(name='Iheanacho', team='LEI', position='FWD', cost=75, score=0.075758)
 Player(name='Rodrigo', team='LEE', position='FWD', cost=65, score=0.069586)
 Player(name='Abraham', team='CHE', position='FWD', cost=65, score=0.066925)
 ```
 
 
### max 2 from a team
```
score: 0.959541, cost: 1000, xP(TP): 3281.63022
GKP(n=2, score=0.105388, cost=115)
 Player(name='Martínez', team='AVL', position='GKP', cost=55, score=0.054386)
 Player(name='Mendy', team='CHE', position='GKP', cost=60, score=0.051002)
DEF(n=5, score=0.280122, cost=290)
 Player(name='Mendy', team='MCI', position='DEF', cost=55, score=0.066246)
 Player(name='Chilwell', team='CHE', position='DEF', cost=60, score=0.060805)
 Player(name='Alexander-Arnold', team='LIV', position='DEF', cost=75, score=0.052823)
 Player(name='Coleman', team='EVE', position='DEF', cost=50, score=0.050404)
 Player(name='Aurier', team='TOT', position='DEF', cost=50, score=0.049844)
MID(n=5, score=0.356106, cost=370)
 Player(name='Gündogan', team='MCI', position='MID', cost=75, score=0.077531)
 Player(name='Son', team='TOT', position='MID', cost=100, score=0.0731)
 Player(name='Pépé', team='ARS', position='MID', cost=75, score=0.070984)
 Player(name='El Ghazi', team='AVL', position='MID', cost=60, score=0.069202)
 Player(name='Willock', team='ARS', position='MID', cost=60, score=0.065289)
FWD(n=3, score=0.217925, cost=225)
 Player(name='Iheanacho', team='LEI', position='FWD', cost=75, score=0.075758)
 Player(name='Cavani', team='MUN', position='FWD', cost=85, score=0.072581)
 Player(name='Rodrigo', team='LEE', position='FWD', cost=65, score=0.069586)
```

### Salah + Fernandes + 2max a team
```
score: 0.928918, cost: 1000, xP(TP): 3176.89956
GKP(n=2, score=0.103275, cost=105)
 Player(name='Martínez', team='AVL', position='GKP', cost=55, score=0.054386)
 Player(name='Meslier', team='LEE', position='GKP', cost=50, score=0.048889)
DEF(n=5, score=0.274006, cost=260)
 Player(name='Stones', team='MCI', position='DEF', cost=55, score=0.066218)
 Player(name='Chilwell', team='CHE', position='DEF', cost=60, score=0.060805)
 Player(name='Coleman', team='EVE', position='DEF', cost=50, score=0.050404)
 Player(name='Aurier', team='TOT', position='DEF', cost=50, score=0.049844)
 Player(name='Phillips', team='LIV', position='DEF', cost=45, score=0.046735)
MID(n=5, score=0.347665, cost=430)
 Player(name='Fernandes', team='MUN', position='MID', cost=120, score=0.078684)
 Player(name='Salah', team='LIV', position='MID', cost=125, score=0.075073)
 Player(name='Torres', team='MCI', position='MID', cost=70, score=0.073246)
 Player(name='El Ghazi', team='AVL', position='MID', cost=60, score=0.069202)
 Player(name='Smith Rowe', team='ARS', position='MID', cost=55, score=0.05146)
FWD(n=3, score=0.203972, cost=205)
 Player(name='Iheanacho', team='LEI', position='FWD', cost=75, score=0.075758)
 Player(name='Rodrigo', team='LEE', position='FWD', cost=65, score=0.069586)
 Player(name='Benteke', team='CRY', position='FWD', cost=65, score=0.058628)
 ```
 
 ### Salah + Fernandes + Kane + 2max a team
 
 ```
 score: 0.889165, cost: 1000, xP(TP): 3040.9443
GKP(n=2, score=0.103275, cost=105)
 Player(name='Martínez', team='AVL', position='GKP', cost=55, score=0.054386)
 Player(name='Meslier', team='LEE', position='GKP', cost=50, score=0.048889)
DEF(n=5, score=0.273675, cost=285)
 Player(name='Stones', team='MCI', position='DEF', cost=55, score=0.066218)
 Player(name='Rüdiger', team='CHE', position='DEF', cost=55, score=0.054386)
 Player(name='Alexander-Arnold', team='LIV', position='DEF', cost=75, score=0.052823)
 Player(name='Coleman', team='EVE', position='DEF', cost=50, score=0.050404)
 Player(name='Aurier', team='TOT', position='DEF', cost=50, score=0.049844)
MID(n=5, score=0.31088, cost=360)
 Player(name='Salah', team='LIV', position='MID', cost=125, score=0.075073)
 Player(name='El Ghazi', team='AVL', position='MID', cost=60, score=0.069202)
 Player(name='Traoré', team='AVL', position='MID', cost=60, score=0.058315)
 Player(name='Jorginho', team='CHE', position='MID', cost=60, score=0.05683)
 Player(name='Smith Rowe', team='ARS', position='MID', cost=55, score=0.05146)
FWD(n=3, score=0.201335, cost=250)
 Player(name='Kane', team='TOT', position='FWD', cost=125, score=0.078495)
 Player(name='Rodrigo', team='LEE', position='FWD', cost=65, score=0.069586)
 Player(name='Fabio Silva', team='WOL', position='FWD', cost=60, score=0.053254)
 ```